# Computational Patterns

Often when writing code we repeat certain patterns, whether we realize it or not.
If you have learned to write list comprehensions, you are taking advantage of a "control pattern".
Often, these patterns are so common that many packages have built in functions to implement them.

Quoting the [toolz documentation](https://toolz.readthedocs.io/en/latest/control.html):

> The Toolz library contains dozens of patterns like map and groupby. Learning a
> core set (maybe a dozen) covers the vast majority of common programming tasks
> often done by hand. A rich vocabulary of core control functions conveys the
> following benefits:
>
> - You identify new patterns
> - You make fewer errors in rote coding
> - You can depend on well tested and benchmarked implementations

The same is true for xarray.


```{seealso}

The concepts covered here, particularly the emphasis on deleting for loops and focusing on large elements of the computation, are very related to the [array programming style](https://en.wikipedia.org/wiki/Array_programming) or paradigm. See this SciPy 2023 tutorial on ["Thinking Like Arrays"](https://github.com/jpivarski-talks/2023-07-11-scipy2023-tutorial-thinking-in-arrays) by Jim Pivarski if you are interested in these ideas.
```

## Motivation / Learning goals

- Learn what high-level computational patterns are available in Xarray.
- Learn that these patterns replace common uses of the `for` loop.
- Identify when you are re-implementing an existing computational pattern.
- Implement that pattern using built-in Xarray functionality.
- Understand the difference between `map` and `reduce`.

## Xarray's high-level patterns

Xarray allows you to leverage dataset metadata to write more readable analysis
code. The metadata is stored with the data; not in your head.

1. Dimension names: `dim="latitude"` instead of `axis=0`
2. Coordinate "labels": or axis tick labels. `data.sel(latitude=45)` instead of
   `data[10]`

Xarray also provides computational patterns that cover many data
analysis tasks.

Xarray provides methods for high-level analysis patterns:

1. `rolling` :
   [Operate on rolling or sliding (fixed length, overlapping) windows of your data e.g. running mean.](https://docs.xarray.dev/en/stable/user-guide/computation.html#rolling-window-operations)
1. `coarsen` :
   [Operate on blocks (fixed length) of your data (downsample).](https://docs.xarray.dev/en/stable/user-guide/computation.html#coarsen-large-arrays)
1. `groupby` :
   [Parse data into groups (using an exact value) and operate on each one (reduce data).](https://docs.xarray.dev/en/stable/groupby.html)
1. `groupby_bins`: [GroupBy after discretizing a numeric (non-exact, e.g. float) variable.](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby_bins.html)
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)
1. `weighted`: [Weight your data before reducing.](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions) 


```{note}
The documentation links in this tutorial point to the DataArray implementations of each function, but they are also available for DataSet objects.
```


### Load example dataset


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# reduce figure size
plt.rcParams["figure.dpi"] = 90
xr.set_options(keep_attrs=True, display_expand_data=False)

da = xr.tutorial.load_dataset("air_temperature", engine="netcdf4").air
monthly = da.resample(time="M").mean()
data = da.isel(time=0)
data.plot();

In [ ]:
da

***

### Identifying high-level computation patterns

*or, when should I use these functions?*

Consider a common use case. We want to complete some "task" for each of "something". The "task" might be a computation (e.g. mean, median, plot). The "something" could be a group of array values (e.g. pixels) or segments of time (e.g. monthly or seasonally).

Often, our solution to this type of problem is to write a for loop. Say we want the average air temperature for each month across the entire domain (all lat and lon values):

In [ ]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
avg_temps = []

# for loop
for mon in months:
    # filter data, split out in to groups
    subset = da[da["time.month"] == mon]
    # do some computation
    avg = subset.mean()
    # append to existing results
    avg_temps.append(avg.item())

print(avg_temps)

This pattern is the GroupBy pattern.


An easy conceptual next step for this example (but still using our for loop) would be to use Xarray's `groupby` function to create an iterator that does the work of grouping our data by month and looping over each month.

In [ ]:
avg_temps = []

for label, group in da.groupby("time.month"):
    avg_temps.append(float(group.mean().data))

print(avg_temps)

Writing a for-loop here is not wrong, but it can quickly become cumbersome if you have a complex function to apply and it will take a while to compute on a large dataset (you may even run out of memory). Parallelizing the computation would take a lot of additional work.

Xarray's functionality instead allows us to do the same computation in one line of code (plus, the computation is optimized and ready to take advantage of parallel compute resources)!

In [ ]:
# note the use of the ellipses here
# for easy comparison to the for loop above
avg_temps = da.groupby("time.month").mean(...)
print(avg_temps.data)

```{note}
:class: dropdown
By default, `da.mean()` (and `df.mean()`) will calculate the mean by reducing your data over all dimensions (unless you specify otherwise using the `dim` kwarg). The default behavior of `.mean()` on a groupby is to calculate the mean over all dimensions of the variable you are grouping by - but not all the dimensions of the object you are operating on. To compute the mean across all dimensions of a groupby, we must specify `...` for all dimensions (or use the `dim` kwarg to specify which dimensions to reduce by).

```

Here we showed an example for computing a mean over a certain period of time (months), which ultimately uses the `GroupBy` function to group together observations with similar characteristics that are scattered through the dataset. The transition from loops to a built-in function is similar for `rolling` and `coarsen` over sequential windows of values (e.g. pixels) instead of "groups" of time.

Read on through this tutorial to learn some of the incredible ways to use Xarray to avoid writing long for-loops and efficiently complete computational analyses on your data.

```{seealso}
:class: dropdown
For a more complex example (identifying flood events - including their start and end date - from rainfall data) illustrating the transition from for loops to high level computation tools, see [this discussion](https://github.com/pydata/xarray/discussions/7641). The [original 40 lines of code](https://github.com/pydata/xarray/discussions/7641#discussion-4976005), including nested for loops, was streamlined into a ~15 line workflow without any loops.
```

***

### Concept refresher: "index space" vs "label space"


In [ ]:
data

In [ ]:
# index space
data[10, :]  # 10th element along the first axis; ¯\_(ツ)_/¯

In [ ]:
# slightly better index space
data.isel(lat=10)  # slightly better, 10th element in latitude

In [ ]:
# "label" space
data.sel(lat=50)  # much better! lat=50°N

In [ ]:
# What I wanted to do
data.sel(lat=50)

# What I had to do (if I wasn't using xarray)
data[10, :]

***

## Xarray provides patterns in both "index space" and "label space"

### Index space

These are sequential windowed operations with a window of a fixed size.

1. `rolling` :
   [Operate on rolling or sliding (fixed length, overlapping) windows of your data e.g. running mean.](https://docs.xarray.dev/en/stable/user-guide/computation.html#rolling-window-operations)
1. `coarsen` :
   [Operate on blocks (fixed length) of your data (downsample).](https://docs.xarray.dev/en/stable/user-guide/computation.html#coarsen-large-arrays)


### Label space

These are windowed operations with irregular windows based on your data. Members of a single group may be non-sequential and scattered through the dataset.

1. `groupby` :
   [Parse data into groups (using an exact value) and operate on each one (reduce data).](https://docs.xarray.dev/en/stable/groupby.html)
1. `groupby_bins`: [GroupBy after discretizing a numeric (non-exact, e.g. float) variable.](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby_bins.html)
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)


add some "loop" versions to show what a user might come up with that could be turned into one of these pattern operations

---

## Index space: windows of fixed width

### Sliding windows of fixed length: [`rolling`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.rolling.html)

- Supports common reductions : `sum`, `mean`, `count`, `std`, `var` etc.
- Returns object of same shape as input
- Pads with NaNs to make this happen
- Supports multiple dimensions

Here's the dataset


In [ ]:
data.plot();

And now smoothed 5 point running mean in lat and lon


In [ ]:
data.rolling(lat=5, lon=5, center=True).mean().plot();

#### Apply an existing numpy-only function with `reduce`

In some cases, we may want to apply a sliding window function using rolling that is not built in to Xarray. In these cases we can still leverage the sliding windows of rolling and apply our own function with [`reduce`](https://docs.xarray.dev/en/stable/generated/xarray.core.rolling.DataArrayRolling.reduce.html).

The `reduce` method on Xarray objects (e.g. DataArray, Dataset) expects a function that can *receive and return plain arrays (e.g. numpy)*, as in each of the "windows" provided by the rolling iterator. This is in contrast to the `map` method on DataArray and Dataset objects, which expects a function that can receive and return Xarray objects.

Here's an example function: [`np.ptp`](https://numpy.org/doc/stable/reference/generated/numpy.ptp.html).


In [ ]:
data.rolling(lat=5, lon=5, center=True).reduce(np.ptp).plot();

```{exercise}
:label: rolling-reduce

Calculate the rolling mean in 5 point bins along both latitude and longitude using
[`rolling(...).reduce`](https://docs.xarray.dev/en/stable/generated/xarray.core.rolling.DataArrayRolling.reduce.html)

```

````{solution} rolling-reduce
:class: dropdown

```python
# exactly equivalent to data.rolling(...).mean()
data.rolling(lat=5, lon=5, center=True).reduce(np.mean).plot();
```

````

#### View the `rolling` operation as a Xarray object with `construct`

In the above examples, we plotted the outputs of our rolling operations. Xarray makes it easy to integrate the outputs from `rolling` directly into the DataArray using the [`construct`](https://docs.xarray.dev/en/stable/generated/xarray.core.rolling.DataArrayRolling.construct.html#xarray.core.rolling.DataArrayRolling.construct) method.

In [ ]:
simple = xr.DataArray(np.arange(10), dims="time", coords={"time": np.arange(10)})
simple

In [ ]:
# adds a new dimension "window"
simple.rolling(time=5, center=True).construct("window")

```{note}
:class: dropdown
Because `.construct()` only returns a "view" (not a copy) of the original data object (i.e. it is not operating "in-place"), in order to "save" the results you would need to rewrite the original object: `simple = simple.rolling(time=5, center=True).construct("window")`.
```

```{exercise}
:label: rolling-construct
Calculate the 5 point running mean in time using `rolling.construct`
```

````{solution} rolling-construct
:class: dropdown

```python
simple.rolling(time=5, center=True).construct("window").mean("window")
```

````


`construct` is clever.

1. It constructs a [**view**](https://numpy.org/doc/stable/user/basics.copies.html) of the original array, so it is memory-efficient.
1. It does something sensible for dask arrays (though generally you want big chunksizes for the dimension you're sliding along).
1. It also works with rolling along multiple dimensions!


#### Advanced: Another `construct` example

This is a 2D rolling example; we need to provide two new dimension names.


In [ ]:
data.rolling(lat=5, lon=5, center=True).construct(lat="lat_roll", lon="lon_roll")

***

### Block windows of fixed length: `coarsen`

For non-overlapping windows or "blocks" use [`coarsen`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.coarsen.html). The syntax is very similar to `rolling`. You will need to specify how you want Xarray to handle the `boundary` if the length of the dimension is not a multiple of the block size.


- Supports common reductions : `sum`, `mean`, `count`, `std`, `var` etc.
- Does **not** return an object of same shape as input
- Allows controls over behaviour at boundaries
- Supports multiple dimensions

In [ ]:
data

In [ ]:
data.plot();

In [ ]:
data.coarsen(lat=5, lon=5, boundary="trim").mean()

In [ ]:
(data.coarsen(lat=5, lon=5, boundary="trim").mean().plot())

#### Coarsen supports `reduce` for custom reductions

```{exercise}
:label: coarsen-reduce
Use `coarsen.reduce` to apply `np.ptp` in 5x5 (lat x lon) point blocks to `data`
```

````{solution} coarsen-reduce
:class: dropdown

```python
data.coarsen(lat=5, lon=5, boundary="trim").reduce(np.mean).plot();
```

````


#### Coarsen supports `construct` for block reshaping and storing outputs

```{tip}
`coarsen.construct` is usually a good alternative to `np.reshape`
```

A simple example splits a 2-year long monthly 1D time series into a 2D array shaped (year x month)


In [ ]:
months = xr.DataArray(
    np.tile(np.arange(1, 13), reps=2),
    dims="time",
    coords={"time": np.arange(1, 25)},
)
months

In [ ]:
# break "time" into two new dimensions: "year", "month"
months.coarsen(time=12).construct(time=("year", "month"))

Note two things:

1. The `time` dimension was also reshaped.
1. The new dimensions `year` and `month` don't have any coordinate labels
   associated with them.

What if the data had say 23 instead of 24 values (`months.isel(time=slice(1, None)`)? In that case we specify a different `boundary` (the default `boundary="exact"` worked above); here we pad to 24 values.


In [ ]:
months.isel(time=slice(1, None)).coarsen(time=12, boundary="pad").construct(time=("year", "month"))

This adds values at the end of the array (see the 'nan' at the end of the time coordinate?), which is not so sensible for this
problem.  We have some control of the padding through the `side` kwarg to `coarsen`. For `side="right"` we get more sensible output.

In [ ]:
months.isel(time=slice(1, None)).coarsen(time=12, boundary="pad", side="right").construct(
    time=("year", "month")
)

Note that `coarsen` pads with NaNs. For more control over padding, use
[DataArray.pad](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.pad.html) explicitly.

In [ ]:
(
    months.isel(time=slice(1, None))
    .pad(time=(1, 0), constant_values=-1)
    .coarsen(time=12)
    .construct(time=("year", "month"))
)

```{note}
The value specified in `.pad` only applies the `fill_value` to the array, not to coordinate variables.
This is why the first value of time in the above example is NaN and not -1.
```

```{exercise}
:label: reshape
Reshape the `time` dimension of the DataArray `monthly` to year x
month and visualize the seasonal cycle for two years at 250°E
```


````{solution} reshape
:class: dropdown

```python
# splits time dimension into year x month
year_month = monthly.coarsen(time=12).construct(time=("year", "month"))

# assign a nice coordinate value for month
year_month["month"] = [
    "jan",
    "feb",
    "mar",
    "apr",
    "may",
    "jun",
    "jul",
    "aug",
    "sep",
    "oct",
    "nov",
    "dec",
]

# assign a nice coordinate value for year
year_month["year"] = [2013, 2014]

# seasonal cycle for two years
year_month.sel(lon=250).plot.contourf(col="year", x="month", y="lat")
```

````




```{exercise}
:label: rolling
Calculate the rolling 4 month average, averaged across years. (This exercise came up during a live lecture).
```


````{solution} rolling
:class: dropdown

1. We first reshape using `coarsen.construct` to add `year` as a new dimension.
2. Apply `rolling` on the month dimension.
3. It turns out that `roll.mean(["year", "month"])` doesn't work. So we use `roll.construct` to get a DataArray with a new dimension `window` and then take the mean over `window` and `year`

```python
reshaped = months.coarsen(time=12).construct(time=("year", "month"))
roll = reshaped.rolling(month=4, center=True)
roll.construct("window").mean(["window", "year"])
```

````

### Summary

Delete your `for` loops. Use `rolling` and `coarsen` for fixed size windowing operations.

1. `rolling` for overlapping windows
1. `coarsen` for non-overlapping windows.

Both provide the usual reductions as methods (`.mean()` and friends), and also
`reduce` and `construct` for custom operations.


***

## Label space "windows" or bins : GroupBy

Sometimes the windows you want are not regularly spaced or even defined by a grid.
For instance, grouping data by month (which have varying numbers of days) or the results of an image classification.
The GroupBy functions are essentially a generalization of `coarsen`: 

- `groupby`: divide data into distinct groups, e.g. climatologies, composites. Works best when the "group identifiers" or "labels" are exact and can be determined using equality (`==`), e.g. characters or integers. Remember that floats are not exact values.
- `groupby_bins`: Use binning operations, e.g. histograms, to group your data.
- `resample`: Specialized implementation of GroupBy specifically for time grouping (so far), allows you to change sampling frequency of dataset.


```{note}
 Both `groupby_bins` and `resample` are implemented as `groupby` with a specific way of constructing group labels. The GroupBy pattern is very flexible!
```


### Deconstructing GroupBy

The GroupBy workflow is commonly called "split-apply-combine".

1. *Split* : break dataset into groups
1. *Apply* : apply an operation, for instance a reduction like `mean`
1. *Combine* : concatenate results from apply step along a new "group" dimension

illustrated in this neat schematic from [Project Pythia](https://foundations.projectpythia.org/core/xarray/computation-masking.html#groupby-split-apply-combine):
<img src="https://foundations.projectpythia.org/_images/xarray-split-apply-combine.jpeg" width=30%>


But really there is a "hidden" first step: *identifying* groups (also called "factorization" or sometimes "binning"). Usually this is the hard part.

In reality the workflow is: "identify groups" → "split into groups" → "apply function" → "combine results".


In [ ]:
# recall our earlier DataArray
da

In [ ]:
# GroupBy returns an iterator that traverses the specified groups, here by month.
# Notice that groupby is clever enough for us to leave out the `.dt` before `.month`
# we would need to specify to access the month data directly, as in `da.time.dt.month`.
da.groupby("time.month")

In [ ]:
# for each group (e.g. the air temperature in a given month for all the years),
# compute the mean
da.groupby("time.month").mean()

Notice that since we have averaged over all the years for each month, our resulting DataArray no longer has a "year" coordinate.

If we want to see how Xarray identifies "groups" for the monthly climatology computation, we can plot our input to `groupby`. GroupBy is clever enough to figure out how many values there are an thus how many groups to make.


In [ ]:
da["time.month"].plot();

Similarly for binning (remember this is useful when the parameter you are binning over is not "exact", like a float),


In [ ]:
data.groupby_bins("lat", bins=[20, 35, 40, 45, 50])

and resampling...


In [ ]:
da.resample(time="M")

```{note}

Resampling is changing the frequency of our data to monthly (for two years), so we have 24 bins. GroupBy is taking the average across all data in the same month for two years, so we have 12 bins.

```

### Constructing group labels

If the automatic group detection doesn't work for your problem then these functions are useful for constructing specific "group labels" in many cases

1. [numpy.digitize](https://numpy.org/doc/stable/reference/generated/numpy.digitize.html)
   for binning
1. [numpy.searchsorted](https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html)
   supports many other data types
1. [pandas.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html)
   supports characters, strings etc.
1. [pandas.cut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html)
   for binning
1. ["Datetime components"](https://docs.xarray.dev/en/stable/user-guide/time-series.html#datetime-components)  of Xarray DataArrays
1. [DataArray.isin](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.isin.html)
1. [scipy.ndimage.label](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.label.html)
1. Do you know of any more? (Send in a pull request to update this list!)



```{tip}
Xarray uses [`pandas.factorize`](https://pandas.pydata.org/docs/reference/api/pandas.factorize.html) for `groupby` and [`pandas.cut`](https://pandas.pydata.org/docs/reference/api/pandas.cut.html) for `groupby_bins`.
```

#### ["Datetime components"](https://docs.xarray.dev/en/stable/user-guide/time-series.html#datetime-components) for creating groups

See a full list
[here](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html?highlight=DatetimeAccessor)

These can be accessed in a few different ways as illustrated below.


In [ ]:
da.time

In [ ]:
da.time.dt.day

In [ ]:
da["time.day"]

In [ ]:
da.time.dt.season

#### Construct and use custom labels

##### Custom seasons with `numpy.isin`.

We want to group over four seasons: `DJF`, `MAM`, `JJAS`, `ON` - this makes physical sense in the Indian Ocean basin.

Start by extracting months.


In [ ]:
month = da.time.dt.month.data
month

Create a new empty array


In [ ]:
myseason = np.full(month.shape, "    ")
myseason

Use `isin` to assign custom seasons,


In [ ]:
myseason[np.isin(month, [12, 1, 2])] = "DJF"
myseason[np.isin(month, [3, 4, 5])] = "MAM"
myseason[np.isin(month, [6, 7, 8, 9])] = "JJAS"
myseason[np.isin(month, [10, 11])] = "ON"

Turn our new seasonal group array into a DataArray.

In [ ]:
myseason_da = da.time.copy(data=myseason)
myseason_da

In [ ]:
(
    # Calculate climatology
    da.groupby(myseason_da)
    .mean()
    # reindex to get seasons in logical order (not alphabetical order)
    .reindex(time=["DJF", "MAM", "JJAS", "ON"])
    .plot(col="time")
)

##### `floor`, `ceil` and `round` on time

Additional functionality in the [datetime accessor](https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html) allows us to effectively "resample" our time data to remove roundoff errors in timestamps.


In [ ]:
da.time

In [ ]:
# remove roundoff error in timestamps
# floor to daily frequency
da.time.dt.floor("D")

##### `strftime` is another powerful option

So useful and so unintuitive that it has its own website: https://strftime.org/

This is useful to avoid merging "Feb-29" and "Mar-01" for a daily climatology


In [ ]:
da.time.dt.strftime("%b-%d")

### Custom reductions with `GroupBy.reduce`

Analogous to `rolling`, `reduce` and `map` apply custom reductions to `groupby_bins` and `resample`.


In [ ]:
(da.groupby("time.month").reduce(np.ptp).plot(col="month", col_wrap=4))

```{tip}
 `map` is for functions that expect and return xarray objects (see also [`Dataset.map`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.map.html)). `reduce` is for functions that expect and return plain arrays (like Numpy or SciPy functions).
```


### Viewing the GroupBy operation on your DataArray or DataSet

GroupBy does *not* provide a `construct` method, because all the groups need not be the same "length" (e.g. months can have 28, 29, 30, or 31 days).

#### Instead looping over groupby objects is possible

Because `groupby` returns an iterator that loops over each group, it is easy to loop over groupby objects. You can also iterate over `rolling` and `coarsen` objects, however this approach is usually quite slow.

Maybe you want to plot data in each group separately:


In [ ]:
for label, group in da.groupby("time.month"):
    print(label)

`group` is a DataArray containing data for all December days (because the last printed `label` value is `12`, so the last `group` value is for December).

In [ ]:
group

Maybe you want a histogram of December temperatures?


In [ ]:
group.plot.hist()

Remember, this example is just to show how you could operate on each group object in a groupby operation. If we wanted to just explore the December (or March) data, we should just filter for it directly:

In [ ]:
da[da["time.month"] == 12].plot.hist()

#### In most cases, avoid a for loop using `map`

`map` enables us to apply functions that expect xarray Datasets or DataArrays. This makes it easy to perform calculations on the grouped data, add the results from each group back to the original object, and avoid having to manually combine results (using concat).


```{tip}
The implementation of `map` *is* a `for` loop. We like `map` because it's cleaner.
```

In [ ]:
def iqr(gb_da, dim):
    """Calculates interquartile range"""
    return (gb_da.quantile(q=0.75, dim=dim) - gb_da.quantile(q=0.25, dim=dim)).rename("iqr")


da.groupby("time.month").map(iqr, dim="time")

***

## Summary

Xarray provides methods for high-level analysis patterns:

1. `rolling` :
   [Operate on rolling (fixed length, overlapping) windows of your data e.g. running mean.](https://docs.xarray.dev/en/stable/user-guide/computation.html#rolling-window-operations)
1. `coarsen` :
   [Operate on blocks (fixed length) of your data (downsample).](https://docs.xarray.dev/en/stable/user-guide/computation.html#coarsen-large-arrays)
1. `groupby` :
   [Parse data into groups (using an exact value) and operate on each one (reduce data).](https://docs.xarray.dev/en/stable/groupby.html)
1. `groupby_bins`: [GroupBy after discretizing a numeric (non-exact, e.g. float) variable.](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby_bins.html)
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)
1. `weighted`: [Weight your data before reducing.](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions)

Xarray also provides a consistent interface to make using those patterns easy:

1. Iterate over the operators (`rolling`, `coarsen`, `groupby`, `groupby_bins`, `resample`).
1. Apply functions that accept numpy-like arrays with `reduce`.
1. Reshape to a new xarray object with `.construct` (`rolling`, `coarsen` only).
1. Apply functions that accept xarray objects with `map` (`groupby`, `groupby_bins`, `resample` only).
